# Assignment 1

Github repo for assignment: https://github.com/brentonjackson/csc-4980/tree/master/Assignment1

I'll be using Python for the assignments in this class, as opposed to Matlab.

## Part A: Fundamentals

Go over camera calibration toolbox and calibrate camera.

It may be worth mentioning that in the [DepthAI documentation](https://docs.luxonis.com/projects/hardware/en/latest/pages/guides/calibration.html), for the nonmodular cameras, they've already been calibrated before shipment so recalibration isn't needed. 

However, I've calibrated the camera by following directions at this link as a learning exercise: https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

Below is the code I used to do this, in Python (skipped in slideshow), and before and after images:

In [3]:
#!/usr/bin/env python3

# Camera calibration for the OAK-D Lite camera (or any camera)


import numpy as np
import cv2 as cv
import glob

# 1. get object points and image points
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7, 3), np.float32)
objp[:, :2] = np.mgrid[0:7, 0:6].T.reshape(-1, 2)
# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.
images = glob.glob('../opencv-samples/left*.jpg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7, 6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (7, 6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()


# 2. calibrate camera
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# 3. undistort image
img = cv.imread('../opencv-samples/left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)
cv.imshow('calibrate result', dst)
cv.waitKey(500)

-1

Distorted image:

![Distorted image](https://github.com/brentonjackson/csc-4980/blob/master/opencv-samples/left12.jpg?raw=true)

Image after calibration and undistortion:

![Undistorted image](calibresult.png)

As you can see from the curved lines in the first image, there was a considerable amount of radial distortion present.

That was fixed in the latter image.

## Part B: Matlab/Python Prototyping 

Write a MATLAB/Python script to find the real world dimensions (e.g. diameter of a ball, side length of a cube) of an object using perspective projection equations. 

Validate using an experiment where you image an object using your camera from a specific distance (choose any distance but ensure you are able to measure it accurately) between the object and camera.